In [0]:
import tweepy
consumer_key = "u5rtT4Esro4z1TCaQlZA1mpun"
consumer_secret = "Ul703KQLO4lOuUrGCLzWffiwDJlbOM3KrIrEX9IYPvcjTvXqPH"
access_token = "748884452613365761-Z79hxJP73f2xAAiRxMajrBR1paK6O0E"
access_token_secret = "HSr2RrJ2KijyQMO4zxkqmCxVaOicZ7SvFhF02A8kpmVnP"

In [0]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 


In [3]:
import csv
import pandas as pd
csvFile = open('tweets.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(["date","tweet"] )
for tweet in tweepy.Cursor(api.search,q="#CWC19",
                           lang="en",
                           since="2019-01-01").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

2019-09-03 03:00:02 #CWC19 #CWC2019 #PAKISTAN #ENGLAND #INDIA #AUSTRALIA #CWC #CricketWorldCup #CricketLIVE #Indians #PAK #AUS #ENG… https://t.co/g8txVdgc3a
2019-09-03 02:58:31 RT @MGCricket: #MitchellStarc broke #GlennMcGrath's record for the most number of wickets taken in a single edition of a #WorldCup this #CW…
2019-09-03 02:06:38 RT @RKSinhaBJP: Feeling proud to see Rishabh Pant, alumni of my school, Indian Public School, Dehradun being a part of Indian Team playing…
2019-09-03 02:00:01 #CWC19 #CWC2019 #PAKISTAN #ENGLAND #INDIA #AUSTRALIA #CWC #CricketWorldCup #CricketLIVE #Indians #PAK #AUS #ENG… https://t.co/sqmnjmscgO
2019-09-03 01:42:48 RT @MGCricket: Which fast bowling #allrounder impressed you the most this #CWC19? #MGCricket #BenStokes #HardikPandya #ThisaraPerera #Moham…
2019-09-03 01:16:29 RT @MGCricket: Which fast bowling #allrounder impressed you the most this #CWC19? #MGCricket #BenStokes #HardikPandya #ThisaraPerera #Moham…
2019-09-03 01:00:02 #CWC19 #CWC2019 #PAKIST

TweepError: ignored

In [0]:
import pandas as pd
df=pd.read_csv("tweets.csv")
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
#nltk.download('punkt')

In [5]:
#Preprocessing
import json
import re
from textblob import TextBlob
import string
from nltk.corpus import stopwords

nltk.download('stopwords')
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def clean_tweets(tweet):
  ps = PorterStemmer()
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(tweet)
  tweet = re.sub(r':', '', tweet)
  tweet = re.sub(r'‚Ä¶', '', tweet)
  tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
  tweet = emoji_pattern.sub(r'', tweet)
  filtered_tweet = [ps.stem(w) for w in word_tokens if not w in stop_words and w not in string.punctuation]
  return ' '.join(filtered_tweet)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
cleaned_tweets = []
for i in range(1,df.shape[0]):
  cleaned_tweets.append(clean_tweets(df.at[i,"tweet"]))
cleaned_tweets

LookupError: ignored

In [7]:
all_tweets = ' '.join(cleaned_tweets)
all_words = all_tweets.split()
all_words

[]

In [8]:
from nltk import ngrams


threegrams = ngrams(all_words, 3)

for grams in threegrams:
  print(grams)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [9]:
#document term matrix
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(cleaned_tweets)
df_term = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
print(df_term)

ValueError: ignored

In [10]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))
  
jaccard_similarity(df.at[1,"tweet"].split(),df.at[0,"tweet"].split())

0.0

In [11]:
import re, math
from collections import Counter
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
  intersection = set(vec1.keys()) & set(vec2.keys())
  numerator = sum([vec1[x] * vec2[x] for x in intersection])
  sum1 = sum([vec1[x]**2 for x in vec1.keys()])
  sum2 = sum([vec2[x]**2 for x in vec2.keys()])
  denominator = math.sqrt(sum1) * math.sqrt(sum2)
  if not denominator:
     return 0.0
  else:
     return float(numerator) / denominator
def get_dice(vec1, vec2):
  intersection = set(vec1.keys()) & set(vec2.keys())
  numerator = 2 *sum([vec1[x] * vec2[x] for x in intersection])
  sum1 = sum([vec1[x]**2 for x in vec1.keys()])
  sum2 = sum([vec2[x]**2 for x in vec2.keys()])
  denominator = sum1 + sum2
  if not denominator:
     return 0.0
  else:
     return float(numerator) / denominator
def text_to_vector(text):
  words = WORD.findall(text)
  return Counter(words)

text1 = df.at[1,"tweet"]
text2 = df.at[0,"tweet"]

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

print("cosine",get_cosine(vector1, vector2))
print("dice",get_dice(vector1,vector2))

cosine 0.15677236033392414
dice 0.15384615384615385
